# Figures associated to immune processing

**You need to tun the notebook 2_ first!!!!**

In [ ]:
from cellassign import assign_cats
import gzip
import itertools as itl
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import seaborn as sns
import triku as tk
import itertools
import matplotlib.pylab as pylab

In [ ]:
from functions import adata_plot_KOvsWT, stat_annot_gene
from functions import dict_WT_KO_colors

In [ ]:
from datetime import date

today = date.today()
today = today.strftime("%Y_%m_%d")

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
params = {'figure.dpi': 200,
          'legend.fontsize': 'large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large', 
         'axes.grid': False}
pylab.rcParams.update(params)
# mpl.style.use('default')

pd.set_option('display.max_columns', None)

## Functions and variables

In [ ]:
%store -r dict_WT_KO_colors
%store -r seed

## Load adata

In [ ]:
adata_all_harmony = sc.read_h5ad('adatas/adata_all_harmony.h5')
adata_imm = adata_all_harmony[adata_all_harmony.obs['merged_cell_type_name'].isin(['APC', 'LYM', 'NEU', 'MC'])]

In [ ]:
sc.pp.filter_genes(adata_imm, min_counts=1)

In [ ]:
sc.pp.pca(adata_imm, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_imm, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_imm, random_state=seed, n_neighbors=int(len(adata_imm) ** 0.5 * 0.5), metric='cosine', use_rep='X_pca_harmony')
tk.tl.triku(adata_imm, use_raw=False)

sc.pp.pca(adata_imm, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_imm, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_imm, random_state=seed, n_neighbors=int(len(adata_imm) ** 0.5 * 0.5), metric='cosine', use_rep='X_pca_harmony')

In [ ]:
sc.tl.umap(adata_imm, min_dist=1)

### Subpopulation annotation

To do the annotation we used the characterization made by Joost et al. 2020. There are some populations that are not appearing (Krt75+ to Krt18+) and we are not sure about their functional relationship

In [ ]:
dict_immune = {  '0: LC': ['Mfge8', 'Cd207', 'Cldn1', 'Pgf', 'Cd74', 'H2-Aa', 'H2-Eb1',  'Nr4a3', 'Il1r2', 'Pkib', 'Grk3', 'Ktn1', 'Pxdc1', 'Dnah2', 'Timd4', 
                                      'Epcam', 'Ccr7', 'St14', 'Mreg', 'Cd40'],
                 '1: CCR7$^+$ LC': ['Fscn1', 'Cd200', 'Il4i1', 'Pkib', 'Wnk1', 'Cd74', 'Cst3', 'Ccr7', 'Bcl2a1a', 'Cacnb3', 'Ccl22', 'Gm13546', 'Serpinb9b', 'Relb', 'Cd40', 'Arl5a'],

                 '2A: Mφ/DCA': ['Pf4', 'C1qb', 'Igf1', 'Grn', 'Dab2', 'C1qc', 'Apoe', 'C1qa', 'Ms4a7', 'Timp2', 'Cd36', 'Mrc1', 'Fxyd2', 'F13a1', 'Rgl1', 'Ednrb', 'Hgsnat', 'Tgfbi', 'Rnase4', 
                                 'Sash1', 'Folr2', 'Gas6', 'Stab1', 'Ccl8', 'Fcgrt', 'Ms4a6c', 'P2rx7'] ,
                '2B: Mφ/DCB': ['Cd209a', 'Cd209d', 'Slc39a14', 'Plbd1', 'Aif1', 'Cd74', 'Clec4e', 'Tgfbi', 'Pdpn', 'Cxcl3', 'Vat1', 'Eps8', 'Tppp3', 
                                     'Procr', 'Tarm1', 'Clec4a3', 'Ccl5'] ,
               '2C: Mφ/DCC': ['Nos2', 'Saa3', 'Arg1', 'Wfdc17', 'Csf1r', 'F10', 'Ly6c2', 'Gsr', 'Anxa5', 'Ptgs2', 'Cxcl3', 'Pid1', 'Serpinb2', 'Csf3', 'Vcan', 'Inhba', 'Ptges', 
                               'F13a1', 'Met', ],
    
                 '3: Act. Neu': ['Acod1', 'Il1f9', 'Csta3', 'Rhov', 'Stfa2l1', 'Ccl3', 'Hcar2', 'Slpi', 'Ccrl2', 'Il1rn', 'Cxcl3', 'G0s2', 
                                      'Stfa2', 'Ptgs2', 'Asprv1', 'Fpr1'],
    
                 '4: Neu': ['Camp', 'Ltf', 'Chil3', 'S100a8', 'S100a9', 'Tinagl1', 'Stfa1', 'Ngp', 'Chil3', 'Cd177', 'Adpgk', 'Syne1', 'Cebpe', 'Hsd11b1', 'Mgst1', 
                                 'Tst', 'Ly6c2', 'Pilrb2', 'Fcnb', 'Rab44', 'Nfe2', 'Cpne3', 'Ldhc', 'Hmgn2', 'Adgrg3'],
               
                 '5: Bas': ['Mcpt8', 'Cyp11a1', 'Cd200r3', 'Ifitm1', 'Ero1l', 'Ccl6', 'Ccl3', 'Lilrb4a', 'Gata2', 'Ccl4', 'Hdc', 'Osm', 'Lat2', 'Tec', 'Rab44', 'Csf2rb2', 
                               'Os9', 'Ier3', 'Aqp9', 'Cd200r4', 'Emilin2', 'Csf1', 'Slpi', 'Hmox1', 'Mboat1', 'Hs3st1', 'Adgrg3', ],
                 
                 '6: BC': ['Vpreb3', 'Cd79a', 'Cd79b', 'Chchd10', 'Ighm',  'Stmn1', 'Igll1', 'Dut', 'Top2a', 'Rasgrp2', 'Myb', 'Ssrp1', 'Gimap6', 'Selenoh', 'Apobec3', 'Smc2',
                             'Cks1b', 'Lgals9', 'Ebf1', 'Csrp2', 'Adgrg1', 'Pou2af1', 'Tcf3', 'Hist1h1a', 'Parp1', 'Lig1', 'Dntt', 'Sox4'],
               
                 '7: PC': ['Iglc3', 'Ms4a1', 'Iglc2', 'Ighm', 'Igkc', 'Cd79a', 'Ly6d', 'Cd79b', 'Tnfrsf13c', 'Iglc1', 'Sdc1', 'Cd74', 
                                  'Txndc16', 'Ighd'],    
                  '8: αβT': ['Tnfrsf18', 'Cxcr6', 'Ccr2', 'Ifngr1', 'Cd52', 'Rora', 'Il7r', 'Aebp2', 'Icos', 'Dgat1', 'Shisa5', 'Grap', 'Cd27', 'Pdcd1', 'Ikzf3', 'Il18r1', 
                             'Gata3', 'Spn', 'Tnfrsf4'],
                  '9: γδT': ['Nkg7', 'Ctsw', 'Cd7', 'Rab4a', 'Ctla2a', 'Rgs2', 'Cd3d', 'Cd3g', 'Fermt2', 'Tcrg-C1', 'Klrk1', 'Trf', 'Itk', 'Ubald2', 'Il2rb', 'Map1b', 'Klrb1b', 'Cd2', 
                              'Trbc2', 'Abcb1b', 'Rasl11b', 'Parp8', 'Tiam1'],
                  '10: MC': ['Cma1', 'Tpsb2', 'Mcpt4', 'Fdx1', 'Hs3st1', 'Cd55', 'Tpsab1', 'Hdc', 'Tph1', 'Cpa3', 'Fgfr1', 'Gata2', 'Slco2b1', 'Vwa5a', 'Il1rl1', 
                                'Cyp11a1', 'Adora3', 'Kit', 'Ap1s3', 'Hexb', 'Creb3l1', 'Mrgprb1', 'Rab27b', 'Penk', 'Hs6st2', 'Adamts9', 'Akap12', 'Slc45a3', 'Mrgprb2', 'Gnai1'], 
                  }



dict_imm_colors = { '0: LC': '#D40055',
                    '1: CCR7$^+$ LC': '#f25a97',
                    '2: Mφ/DC': '#4D0765',
                    '3: Act. Neu': '#D3BC5F',
                    '4: Neu': '#A5901A',
                    '5: Bas': '#A05A2C',
                    '6: BC': '#4c708a',
                    '7: PC': '#00171F',
                    '8: αβT': '#0086b9',
                    '9: γδT': '#99DCF5',
                    '10: MC': '#FF9955',
                     }


In [ ]:
sc.tl.leiden(adata_imm, resolution=4)
assign_cats(adata_imm, dict_cats=dict_immune, min_score=0.4, quantile_gene_sel=0.6, key_added='subtype')

adata_imm.obs['subtype'] = [i  if i[0] != '2' else '2: Mφ/DC' for i in adata_imm.obs['subtype']]


adata_imm.obs['subtype_name'] = [i.split(': ')[1] for i in adata_imm.obs['subtype']]
adata_imm.obs['subtype_name'] = adata_imm.obs['subtype_name'].astype('category')
adata_imm.obs['subtype_name'].cat.reorder_categories([i.split(': ')[1] for i in dict_imm_colors.keys()], inplace=True)

adata_imm.obs['subtype_number'] = [i.split(': ')[0] for i in adata_imm.obs['subtype']]
adata_imm.obs['subtype_number'] = adata_imm.obs['subtype_number'].astype('category')
adata_imm.obs['subtype_number'].cat.reorder_categories([i.split(': ')[0] for i in dict_imm_colors.keys()], inplace=True)


adata_imm.uns['subtype_colors'] = list(dict_imm_colors.values())
adata_imm.uns['subtype_name_colors'] = list(dict_imm_colors.values())
adata_imm.uns['subtype_number_colors'] = list(dict_imm_colors.values())

## UMAP of fibroblast subpopulations and batch

#### Supp Fig to Fig 3

In [ ]:
adata_imm_WT = adata_imm[adata_imm.obs['batch'].isin(['WT1', 'WT2'])]
adata_imm_WT.uns['batch_colors'] = [dict_WT_KO_colors['WT1'], dict_WT_KO_colors['WT2'], ]
adata_imm_KO = adata_imm[adata_imm.obs['batch'].isin(['KO1', 'KO2'])]
adata_imm_KO.uns['batch_colors'] = [dict_WT_KO_colors['KO1'], dict_WT_KO_colors['KO2'], ]

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8, 4))
s = 18

sc.pl.umap(adata_imm, color='subtype_number', s=s, ax = axs, 
           legend_loc='on data', frameon=False, show=False, 
           legend_fontoutline=4, legend_fontsize=14)
sc.pl.umap(adata_imm, color='subtype', s=s, ax = axs, 
           frameon=False, show=False)

title_obj = axs.set_title('') 

plt.tight_layout()

# plt.savefig(f'results/figures/F3/makers_FB_closeup_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)
# plt.savefig(f'results/figures/F3/Supp_Fig3A_closeup_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 7))
s = 18


sc.pl.umap(adata_imm_WT, color='batch', s=s, ax = axs.ravel()[0], 
           frameon=False, show=False, na_in_legend=False)


sc.pl.umap(adata_imm_WT, color='subtype_number', s=s, ax=axs.ravel()[1], 
           legend_loc='on data', frameon=False, show=False, legend_fontoutline=4, legend_fontsize=14, 
           na_in_legend=False, title='condition')
sc.pl.umap(adata_imm_WT, color='subtype', s=s, ax = axs.ravel()[1], 
           frameon=False, show=False, na_in_legend=False, title='FB type')



sc.pl.umap(adata_imm_KO, color='batch', s=s, ax = axs.ravel()[2], 
           frameon=False, show=False, na_in_legend=False, title='')


sc.pl.umap(adata_imm_KO, color='subtype_number', s=s, ax=axs.ravel()[3], 
           legend_loc='on data', frameon=False, show=False, legend_fontoutline=4, legend_fontsize=14, 
           na_in_legend=False, title='')


plt.tight_layout()

# plt.savefig(f'results/figures/F3/makers_FB_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)
# plt.savefig(f'results/figures/F3/Supp_Fig3A_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)

#### Fig 3A

In [ ]:
selected_cell_types = list(dict_fbs_colors_nan.keys())

In [ ]:
selected_cell_types

In [ ]:
adata_fb.obs['subtype_nan'] = [i if i in selected_cell_types else np.NaN for i in adata_fb.obs['subtype']]
adata_fb.obs['subtype_nan'] = adata_fb.obs['subtype_nan'].astype('category')
adata_fb.obs['subtype_nan'].cat.reorder_categories(selected_cell_types, inplace=True)

adata_fb.obs['subtype_nan_name'] = [i.split(': ')[1] if i in selected_cell_types else np.NaN for i in adata_fb.obs['subtype_nan']]
adata_fb.obs['subtype_nan_name'] = adata_fb.obs['subtype_nan_name'].astype('category')
adata_fb.obs['subtype_nan_name'].cat.reorder_categories([i.split(': ')[1] for i in selected_cell_types], inplace=True)

adata_fb.obs['subtype_nan_number'] = [i.split(': ')[0] if i in selected_cell_types else np.NaN for i in adata_fb.obs['subtype_nan']]
adata_fb.obs['subtype_nan_number'] = adata_fb.obs['subtype_nan_number'].astype('category')
adata_fb.obs['subtype_nan_number'].cat.reorder_categories([i.split(': ')[0] for i in selected_cell_types], inplace=True)


adata_fb.uns['subtype_nan_colors'] = list(dict_fbs_colors_nan.values())
adata_fb.uns['subtype_nan_name_colors'] = list(dict_fbs_colors_nan.values())
adata_fb.uns['subtype_nan_number_colors'] = list(dict_fbs_colors_nan.values())

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8, 4))
s = 12

sc.pl.umap(adata_fb, color='subtype_nan_number', s=s, ax = axs, 
           legend_loc='on data', frameon=False, show=False, 
           legend_fontoutline=4, legend_fontsize=14, na_in_legend=False)
sc.pl.umap(adata_fb, color='subtype_nan', s=s, ax = axs, 
           frameon=False, show=False, na_in_legend=False)

title_obj = axs.set_title('') 

plt.tight_layout()

plt.savefig(f'results/figures/F3/makers_FB_nan_closeup_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)
plt.savefig(f'results/figures/F3/Fig3A_closeup_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)

In [ ]:
# We repeat the variable naming here (it was done before) because we need to add nan vars to the new adata_fb_WT and adata_fb_KO
adata_fb_WT = adata_fb[adata_fb.obs['batch'].isin(['WT1', 'WT2'])]
adata_fb_WT.uns['subtype_nan_colors'] = adata_fb.uns['subtype_nan_colors']
adata_fb_WT.uns['subtype_nan_number_colors'] = adata_fb.uns['subtype_nan_number_colors']
adata_fb_WT.uns['batch_colors'] = [dict_WT_KO_colors['WT1'], dict_WT_KO_colors['WT2']]

adata_fb_KO = adata_fb[adata_fb.obs['batch'].isin(['KO1', 'KO2'])]
adata_fb_KO.uns['subtype_nan_colors'] = adata_fb.uns['subtype_nan_colors']
adata_fb_KO.uns['subtype_nan_number_colors'] = adata_fb.uns['subtype_nan_number_colors']
adata_fb_KO.uns['batch_colors'] = [dict_WT_KO_colors['KO1'], dict_WT_KO_colors['KO2']]

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 7))
s = 18


sc.pl.umap(adata_fb_WT, color='batch', s=s, ax = axs.ravel()[0], 
           frameon=False, show=False, na_in_legend=False)


sc.pl.umap(adata_fb_WT, color='subtype_nan_number', s=s, ax=axs.ravel()[1], 
           legend_loc='on data', frameon=False, show=False, legend_fontoutline=4, legend_fontsize=14, 
           na_in_legend=False, title='condition')
sc.pl.umap(adata_fb_WT, color='subtype_nan', s=s, ax = axs.ravel()[1], 
           frameon=False, show=False, na_in_legend=False, title='FB type')



sc.pl.umap(adata_fb_KO, color='batch', s=s, ax = axs.ravel()[2], 
           frameon=False, show=False, na_in_legend=False, title='')


sc.pl.umap(adata_fb_KO, color='subtype_nan_number', s=s, ax=axs.ravel()[3], 
           legend_loc='on data', frameon=False, show=False, legend_fontoutline=4, legend_fontsize=14, 
           na_in_legend=False, title='')


plt.tight_layout()

# plt.savefig(f'results/figures/F3/makers_FB_nan_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)
# plt.savefig(f'results/figures/F3/Fig3A_{date.today().strftime("%Y_%m_%d")}.png', dpi=300)

In [ ]:
dict_immune = {  '0: LC': ['Mfge8', 'Cd207', 'Cldn1', 'Pgf', 'Cd74', 'H2-Aa', 'H2-Eb1',  'Nr4a3', 'Il1r2', 'Pkib', 'Grk3', 'Ktn1', 'Pxdc1', 'Dnah2', 'Timd4', 
                                      'Epcam', 'Ccr7', 'St14', 'Mreg', 'Cd40'],
                 '1: CCR7$^+$ LC': ['Fscn1', 'Cd200', 'Il4i1', 'Pkib', 'Wnk1', 'Cd74', 'Cst3', 'Ccr7', 'Bcl2a1a', 'Cacnb3', 'Ccl22', 'Gm13546', 'Serpinb9b', 'Relb', 'Cd40', 'Arl5a'],

                 '2: Mφ/DC': ['Pf4', 'C1qb', 'Igf1', 'Grn', 'Dab2', 'C1qc', 'Apoe', 'C1qa', 'Ms4a7', 'Timp2', 'Cd36', 'Mrc1', 'Fxyd2', 'F13a1', 'Rgl1', 'Ednrb', 'Hgsnat', 'Tgfbi', 'Rnase4', 
                                 'Sash1', 'Folr2', 'Gas6', 'Stab1', 'Ccl8', 'Fcgrt', 'Ms4a6c', 'P2rx7'] + ['Cd209a', 'Cd209d', 'Slc39a14', 'Plbd1', 'Aif1', 'Cd74', 'Clec4e', 'Tgfbi', 'Pdpn', 'Cxcl3', 'Vat1', 'Eps8', 'Tppp3', 
                                     'Procr', 'Tarm1', 'Clec4a3', 'Ccl5'] + ['Nos2', 'Saa3', 'Arg1', 'Wfdc17', 'Csf1r', 'F10', 'Ly6c2', 'Gsr', 'Anxa5', 'Ptgs2', 'Cxcl3', 'Pid1', 'Serpinb2', 'Csf3', 'Vcan', 'Inhba', 'Ptges', 
                               'F13a1', 'Met', ],
    
                 '3: Act. Neu': ['Acod1', 'Il1f9', 'Csta3', 'Rhov', 'Stfa2l1', 'Ccl3', 'Hcar2', 'Slpi', 'Ccrl2', 'Il1rn', 'Cxcl3', 'G0s2', 
                                      'Stfa2', 'Ptgs2', 'Asprv1', 'Fpr1'],
    
                 '4: Neu': ['Camp', 'Ltf', 'Chil3', 'S100a8', 'S100a9', 'Tinagl1', 'Stfa1', 'Ngp', 'Chil3', 'Cd177', 'Adpgk', 'Syne1', 'Cebpe', 'Hsd11b1', 'Mgst1', 
                                 'Tst', 'Ly6c2', 'Pilrb2', 'Fcnb', 'Rab44', 'Nfe2', 'Cpne3', 'Ldhc', 'Hmgn2', 'Adgrg3'],
               
                 '5: Basophil': ['Mcpt8', 'Cyp11a1', 'Cd200r3', 'Ifitm1', 'Ero1l', 'Ccl6', 'Ccl3', 'Lilrb4a', 'Gata2', 'Ccl4', 'Hdc', 'Osm', 'Lat2', 'Tec', 'Rab44', 'Csf2rb2', 
                               'Os9', 'Ier3', 'Aqp9', 'Cd200r4', 'Emilin2', 'Csf1', 'Slpi', 'Hmox1', 'Mboat1', 'Hs3st1', 'Adgrg3', ],
                 
                 '6: B cell': ['Vpreb3', 'Cd79a', 'Cd79b', 'Chchd10', 'Ighm',  'Stmn1', 'Igll1', 'Dut', 'Top2a', 'Rasgrp2', 'Myb', 'Ssrp1', 'Gimap6', 'Selenoh', 'Apobec3', 'Smc2',
                             'Cks1b', 'Lgals9', 'Ebf1', 'Csrp2', 'Adgrg1', 'Pou2af1', 'Tcf3', 'Hist1h1a', 'Parp1', 'Lig1', 'Dntt', 'Sox4'],
               
                 '7: Plasma cell': ['Iglc3', 'Ms4a1', 'Iglc2', 'Ighm', 'Igkc', 'Cd79a', 'Ly6d', 'Cd79b', 'Tnfrsf13c', 'Iglc1', 'Sdc1', 'Cd74', 
                                  'Txndc16', 'Ighd'],    
                  '8: αβT': ['Tnfrsf18', 'Cxcr6', 'Ccr2', 'Ifngr1', 'Cd52', 'Rora', 'Il7r', 'Aebp2', 'Icos', 'Dgat1', 'Shisa5', 'Grap', 'Cd27', 'Pdcd1', 'Ikzf3', 'Il18r1', 
                             'Gata3', 'Spn', 'Tnfrsf4'],
                  '9: γδT': ['Nkg7', 'Ctsw', 'Cd7', 'Rab4a', 'Ctla2a', 'Rgs2', 'Cd3d', 'Cd3g', 'Fermt2', 'Tcrg-C1', 'Klrk1', 'Trf', 'Itk', 'Ubald2', 'Il2rb', 'Map1b', 'Klrb1b', 'Cd2', 
                              'Trbc2', 'Abcb1b', 'Rasl11b', 'Parp8', 'Tiam1'],
                  '10: Mast cell': ['Cma1', 'Tpsb2', 'Mcpt4', 'Fdx1', 'Hs3st1', 'Cd55', 'Tpsab1', 'Hdc', 'Tph1', 'Cpa3', 'Fgfr1', 'Gata2', 'Slco2b1', 'Vwa5a', 'Il1rl1', 
                                'Cyp11a1', 'Adora3', 'Kit', 'Ap1s3', 'Hexb', 'Creb3l1', 'Mrgprb1', 'Rab27b', 'Penk', 'Hs6st2', 'Adamts9', 'Akap12', 'Slc45a3', 'Mrgprb2', 'Gnai1'], 
                  }



dict_imm_colors = { '0: LC': '#D40055',
                    '1: CCR7$^+$ LC': '#8E0C6F',
                    '2: Mφ/DC': '#4D0765',
                    '3: Act. Neu': '#D3BC5F',
                    '4: Neu': '#A5901A',
                    '5: Basophil': '#A05A2C',
                    '6: B cell': '#229EB2',
                    '7: Plasma cell': '#20668D',
                    '8: αβT': '#77ECF1',
                    '9: γδT': '#702A17',
                    '10: Mast cell': '#FF9955',
                     }


In [ ]:
sc.tl.leiden(adata_imm, resolution=4)
assign_cats(adata_imm, dict_cats=dict_immune, min_score=0.4, quantile_gene_sel=0.6, key_added='subtype')

adata_imm.uns['subtype_colors'] = list(dict_imm_colors.values())

## UMAP of fibroblast subpopulations and batch

In [ ]:
sc.tl.umap(adata_imm, min_dist=1)
sc.pl.umap(adata_imm, color=['batch', 'subtype'], ncols=2, frameon=False, cmap=magma)

In [ ]:
sc.tl.umap(adata_imm, min_dist=1)
sc.pl.umap(adata_imm, color=['batch', 'subtype', 'Klrb1c'], ncols=2, frameon=False, cmap=magma)